In [97]:
import os
import pandas as pd
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.models import Sequential
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from numpy import inf

In [127]:
def multi_merge(left,right,*args):
    start = args[0]
    for i in range(1,len(args)):
        start = start.merge(args[i], how = 'left', left_on = left, right_on = right)
    return start

def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type not in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)


def multi_merge(left,right,*args):
    start = args[0]
    for i in range(1,len(args)):
        start = start.merge(args[i], how = 'left', left_on = left, right_on = right)
    return start

def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type not in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

def clean_data(df,year):
    column_list = list(df.columns)
    A_list = [column for column in column_list if re.match(r'^[A][0-9]{2,10}',column) or re.match(r'^[a][0-9]{2,10}',column)]
    N_list = [column for column in column_list if re.match(r'^[N][0-9]{2,10}',column) or re.match(r'^[n][0-9]{2,10}',column)]
    O_list = [column for column in column_list if (column not in A_list and column not in N_list and column not in ['STATE','STATEFIPS','state','statefips'])]
    if 'n1' in O_list:
        N1 = 'n1'
    else:
        N1 = 'N1'
    special = []
    pair = []
    for name in A_list:
        if 'N' + name[1:] not in N_list and 'n' + name[1:] not in N_list:
            special.append(name)
        else:
            pair.append(name)
            if 'n' + name[1:] in N_list:
                pair.append('n' + name[1:])
            else:
                pair.append('N' + name[1:])
    special = O_list + special
    pair = pair + ['zipcode',N1]
    df_special = df.loc[:,special]
    df_pair = df.loc[:,pair]
    for column in list(df_special.columns):
        if column != 'zipcode':
            df_special[column] = df_special[column]/df_special[N1]
    df_special = df_special.groupby(by='zipcode').agg('mean')
    #print(df_special)
    for name in N_list:
        if name[0] == 'N':
            A_name = 'A' + name[1:]
        else:
            A_name = 'a' + name[1:]
        df_pair[A_name] = df_pair[A_name]/df_pair[name]
        df_pair[name] = df_pair[name]/df_pair[N1]
    #print(df_pair)
    df_pair = df_pair.groupby(by='zipcode').mean().drop(N1,axis=1)
    #print(df_pair)
    df_result = df_special.merge(df_pair,how='inner',left_on='zipcode',right_on='zipcode').drop([N1,'agi_stub'],axis=1)
    #print(df_pair.shape)
    #print(df_special.shape)
    #print(df_result.shape)
    result_name_list = [name  if name == 'zipcode' else name + '_' + year for name in list(df_result.columns)]
    df_result.columns = result_name_list
    return df_result

# Train Dataset

In [182]:
# read train/test dataset 
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_detail = pd.read_csv("./16zpallagi.csv")

# filter out 00000 and 99999
df_detail = df_detail.loc[(df_detail['zipcode'] != 0) & (df_detail['zipcode'] != 99999)].reset_index(drop=True)
df_detail.shape

(179184, 147)

In [183]:
df_detail.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,35004,1,1510,990,220,280,700,2130,...,730,664,0,0,0,0,120,109,1290,2692
1,1,AL,35004,2,1410,660,360,340,630,2660,...,1170,3079,0,0,0,0,210,302,1190,3223
2,1,AL,35004,3,950,330,470,140,440,2080,...,920,4988,0,0,0,0,200,420,740,2000
3,1,AL,35004,4,650,120,470,50,320,1690,...,640,5504,0,0,0,0,140,385,500,1535
4,1,AL,35004,5,630,50,560,20,330,1800,...,620,10323,0,0,0,0,230,1030,390,1351


## Merge new dataset

In [184]:
df_business = pd.read_csv("zbp16totals.txt",delimiter=",")
df_business = df_business.iloc[:, np.r_[0,4,6,8,9]]
df_business = df_business.rename(columns={"zip":"zipcode"})
df_business.head()

,zipcode,emp,qp1,ap,est
0,501,0,0,0,2
1,1001,9127,88685,376492,472
2,1002,18894,109167,472633,564
3,1003,288,2371,10461,21
4,1004,193,1335,5915,14


In [185]:
df_detail = df_detail.merge(df_business,how="left",on="zipcode")
df_detail.shape

(179184, 151)

In [186]:
df_detail.head()

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N85300,A85300,N11901,A11901,N11902,A11902,emp,qp1,ap,est
0,1,AL,35004,1,1510,990,220,280,700,2130,...,0,0,120,109,1290,2692,3067.0,25615.0,104969.0,189.0
1,1,AL,35004,2,1410,660,360,340,630,2660,...,0,0,210,302,1190,3223,3067.0,25615.0,104969.0,189.0
2,1,AL,35004,3,950,330,470,140,440,2080,...,0,0,200,420,740,2000,3067.0,25615.0,104969.0,189.0
3,1,AL,35004,4,650,120,470,50,320,1690,...,0,0,140,385,500,1535,3067.0,25615.0,104969.0,189.0
4,1,AL,35004,5,630,50,560,20,330,1800,...,0,0,230,1030,390,1351,3067.0,25615.0,104969.0,189.0


## 1. Data Cleaning

In [187]:
import re
column_list = list(df_detail.columns)
A_list = [column for column in column_list if re.match(r'^[A][0-9]{2,10}',column)] # only have columns with "A..."
N_list = [column for column in column_list if re.match(r'^[N][0-9]{2,10}',column)] # only have columns with "N..."
O_list = [column for column in column_list if (column not in A_list and column not in N_list)] # other columns

special = []
for name in A_list:
    if 'N' + name[1:] not in N_list:
        special.append(name)
# Column1: zipcode column
df_zipcode = df_detail.loc[:,'zipcode']
# pair A... and N...
df_detail_pair = df_detail.iloc[:,18:].drop(special+['SCHF'],axis=1)
df_detail_pair = pd.concat([df_zipcode,df_detail_pair],axis = 1)
for name in N_list:
    A_name = 'A' + name[1:]
    df_detail_pair[A_name] = df_detail_pair[A_name]*df_detail_pair[name]

In [188]:
# Column2: for each zipcode, Total of amount / total of returns
df_detail_pair = df_detail_pair.groupby('zipcode').sum()
for name in N_list:
    A_name = 'A' + name[1:]
    df_detail_pair[A_name] = df_detail_pair[A_name]/df_detail_pair[name]
df_feature_1 = df_detail_pair.loc[:,['A' + name[1:] for name in N_list]].fillna(0)
df_feature_1.head()

,A02650,A00200,A00300,A00600,A00650,A00700,A00900,A01000,A01400,A01700,...,A59720,A11070,A10960,A11560,A06500,A10300,A85530,A85300,A11901,A11902
zipcode,,,,,,,,,,,,,,,,,,,,,
1001,81330.952116,64046.144595,515.501639,935.430108,692.549133,294.103960,2734.028302,1053.503876,2653.745763,8489.688995,...,872.860759,283.943396,102.039216,30.818182,8849.053237,9130.414365,61.0,89.0,747.731544,3429.103933
1002,112762.510011,68397.574440,1365.477273,7195.936170,5828.594855,491.852041,9359.559633,7644.710884,5950.027027,17160.873832,...,951.927711,262.408163,143.475410,75.666667,17565.600846,18141.400534,478.0,913.0,2165.102679,3328.300000
1003,1178.882353,1140.823529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,24.000000,0.000000,50.571429,52.857143,0.0,0.0,0.000000,127.000000
1005,22489.105485,18147.203883,101.540230,255.414634,190.341463,79.781818,1259.212121,431.709677,595.120000,2172.040816,...,205.100000,81.692308,33.642857,25.000000,2333.698925,2440.118557,0.0,0.0,232.511111,942.876344
1007,90681.472296,75133.359133,430.725076,1191.135294,893.522013,396.790698,3282.663793,1659.638462,1956.922078,7871.321212,...,591.716981,157.342857,137.808511,43.636364,11473.818482,11726.790476,160.0,121.0,1088.131387,2862.419795


In [189]:
# column3: 'A00100', 'A00101'
df_special_feature_1 = df_detail.loc[:,special+['zipcode','N1']]
df_special_feature_1 = df_special_feature_1.groupby(by='zipcode').apply(lambda x: pd.Series({'avg_agi':sum(x['N1']*x[special[0]])/sum(x['N1']),'avg_item_r':sum(x['N1']*x[special[1]])/sum(x['N1'])}))
df_special_feature_1.head()

,avg_agi,avg_item_r
zipcode,,
1001,80337.743875,37501.525612
1002,110209.094837,76528.651212
1003,1144.470588,0.000000
1005,22167.666667,10815.708861
1007,89435.886544,60582.518470


In [190]:
df_special_feature_2 = df_detail.iloc[:,:18]
for column in list(df_special_feature_2.columns)[5:]:
    df_special_feature_2[column] = df_special_feature_2[column]/df_special_feature_2['N1']
df_special_feature_2 = df_special_feature_2.fillna(0).drop(['STATEFIPS','STATE','agi_stub'],axis=1).groupby(by='zipcode').agg('mean')
df_special_feature_2.head()

,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,VITA_EIC,RAL,RAC,ELDERLY
zipcode,,,,,,,,,,,,,,
1001,1496.666667,0.434868,0.479333,0.076106,0.606140,1.977950,0.541338,0.012041,0.002372,0.010261,0.000000,0.002990,0.073244,0.299588
1002,1581.666667,0.426821,0.483702,0.075928,0.543996,1.945985,0.512028,0.021110,0.013754,0.007356,0.004028,0.002014,0.057039,0.397310
1003,34.000000,0.400000,0.000000,0.000000,0.026667,0.493333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1005,395.000000,0.349221,0.563934,0.072222,0.588392,2.197465,0.666593,0.000000,0.000000,0.000000,0.000000,0.000000,0.095656,0.234669
1007,1263.333333,0.371875,0.535202,0.074259,0.549149,2.118798,0.646966,0.010649,0.001536,0.009113,0.000000,0.002304,0.067025,0.274557


In [191]:
df_feature_3 = df_detail.iloc[:,147:]
#df_feature_3 = zscore_transformation(df_feature_3)
df_feature_3["zipcode"] = df_detail["zipcode"]
df_feature_3 = df_feature_3.groupby(by='zipcode').agg('mean')
df_feature_3.head()

,emp,qp1,ap,est
zipcode,,,,
1001,9127.0,88685.0,376492.0,472.0
1002,18894.0,109167.0,472633.0,564.0
1003,288.0,2371.0,10461.0,21.0
1005,1089.0,8439.0,34815.0,95.0
1007,1989.0,13957.0,63860.0,229.0


In [192]:
df_full_detail = multi_merge('zipcode','zipcode',df_special_feature_2,df_special_feature_1,df_feature_1,df_feature_3)
df_full_detail.head()

,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,...,A06500,A10300,A85530,A85300,A11901,A11902,emp,qp1,ap,est
zipcode,,,,,,,,,,,,,,,,,,,,,
1001,1496.666667,0.434868,0.479333,0.076106,0.606140,1.977950,0.541338,0.012041,0.002372,0.010261,...,8849.053237,9130.414365,61.0,89.0,747.731544,3429.103933,9127.0,88685.0,376492.0,472.0
1002,1581.666667,0.426821,0.483702,0.075928,0.543996,1.945985,0.512028,0.021110,0.013754,0.007356,...,17565.600846,18141.400534,478.0,913.0,2165.102679,3328.300000,18894.0,109167.0,472633.0,564.0
1003,34.000000,0.400000,0.000000,0.000000,0.026667,0.493333,0.000000,0.000000,0.000000,0.000000,...,50.571429,52.857143,0.0,0.0,0.000000,127.000000,288.0,2371.0,10461.0,21.0
1005,395.000000,0.349221,0.563934,0.072222,0.588392,2.197465,0.666593,0.000000,0.000000,0.000000,...,2333.698925,2440.118557,0.0,0.0,232.511111,942.876344,1089.0,8439.0,34815.0,95.0
1007,1263.333333,0.371875,0.535202,0.074259,0.549149,2.118798,0.646966,0.010649,0.001536,0.009113,...,11473.818482,11726.790476,160.0,121.0,1088.131387,2862.419795,1989.0,13957.0,63860.0,229.0


In [198]:
# normalization
def normalization(x):
    min_x = min(x)
    max_x = max(x)
    return np.array([(i-min_x)/(max_x-min_x) for i in x])

def zscore_transformation(df):
    for name in df.columns:
        if (is_numeric_dtype(df[name])):
            df[name] = zscore(df[name])
    return df.dropna(axis=1).dropna(axis=0)

def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    

for name in list(df_full_detail.columns)[2:78]:
        df_full_detail[name] = zscore(df_full_detail[name])
        
encode_numeric_zscore(df_full_detail,"emp",mean=None, sd=None)
encode_numeric_zscore(df_full_detail,"est",mean=None, sd=None)
df_full_detail = df_full_detail.drop(["qp1","ap"],axis=1)

In [199]:
df_train.head()

,id,zipcode,score
0,2,8810,11
1,3,28129,4
2,4,79915,25
3,5,21085,10
4,6,62882,1


In [200]:
df_full_detail.head()

,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,...,A10960,A11560,A06500,A10300,A85530,A85300,A11901,A11902,emp,est
zipcode,,,,,,,,,,,,,,,,,,,,,
1001,1496.666667,0.434868,0.004346,0.035355,0.607133,0.083044,-0.274918,0.314364,-0.123866,0.812419,...,0.208351,0.083241,-0.004839,-0.000791,-0.166880,-0.148189,-0.106294,0.158705,0.639908,0.516977
1002,1581.666667,0.426821,0.040088,0.031596,0.112454,-0.000441,-0.422519,0.964526,0.861449,0.443792,...,0.524471,1.096606,0.265020,0.275680,0.228821,0.130776,0.361554,0.135777,1.849339,0.737564
1003,34.000000,0.400000,-3.917232,-1.571881,-4.005592,-3.794371,-3.001041,-0.548755,-0.329249,-0.489604,...,-0.387016,-0.613106,-0.277236,-0.279304,-0.224765,-0.178320,-0.353106,-0.592387,-0.454610,-0.564379
1005,395.000000,0.349221,0.696493,-0.046657,0.465854,0.656358,0.355856,-0.548755,-0.329249,-0.489604,...,-0.313450,-0.048223,-0.206551,-0.206059,-0.224765,-0.178320,-0.276359,-0.406809,-0.355424,-0.386950
1007,1263.333333,0.371875,0.461430,-0.003652,0.153468,0.450900,0.257015,0.214602,-0.196270,0.666696,...,0.481238,0.372872,0.076422,0.078870,-0.072937,-0.137355,0.006066,0.029808,-0.243978,-0.065660


In [201]:
# train data with 2016 and test data with 2016
df_train = multi_merge('zipcode','zipcode',df_train,df_full_detail).drop(['id'],axis=1)
df_test = multi_merge('zipcode','zipcode',df_test,df_full_detail).drop(['id'],axis=1)

In [202]:
df_train.head()

,zipcode,score,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,...,A10960,A11560,A06500,A10300,A85530,A85300,A11901,A11902,emp,est
0,8810,11,638.333333,0.370985,0.423846,0.097313,0.495901,0.726038,0.889620,0.255849,...,0.015545,0.155135,0.092511,0.086426,0.227872,-0.111625,0.020340,-0.231348,0.566725,0.157325
1,28129,4,406.666667,0.263607,1.080122,-0.068464,1.100166,0.853657,-0.010636,1.060571,...,-0.304368,0.109944,-0.221431,-0.221210,-0.224765,-0.178320,-0.268795,-0.383556,-0.358148,-0.355780
2,79915,25,2585.000000,0.267917,-0.530629,1.896657,-0.499604,1.105949,1.692514,0.531743,...,2.459090,1.317320,-0.151427,-0.116272,-0.224765,-0.178320,-0.188733,3.771184,0.675942,1.070843
3,21085,10,1448.333333,0.361274,0.268809,0.505977,-0.072434,0.338366,0.037674,0.723498,...,0.105557,0.109944,0.036310,0.034517,-0.134617,-0.141418,-0.059651,0.170816,-0.122379,0.071008
4,62882,1,158.333333,0.229243,-0.048833,-0.005400,-0.072534,-0.161518,-0.109550,0.304627,...,-0.455678,-0.613106,-0.263145,-0.264522,-0.224765,-0.178320,-0.317968,-0.464303,-0.461916,-0.521220


In [203]:
df_test.head()

,zipcode,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,VITA,...,A10960,A11560,A06500,A10300,A85530,A85300,A11901,A11902,emp,est
0,61088,483.333333,0.314444,0.798960,0.232770,0.423123,0.711101,0.531600,-0.548755,-0.329249,...,-0.161758,0.335897,-0.150339,-0.151698,-0.193450,-0.167486,-0.218706,-0.352575,-0.395916,-0.365371
1,30410,98.333333,0.218910,-0.464642,0.684367,-0.058319,0.182208,0.788796,-0.548755,-0.329249,...,-0.333613,-0.613106,-0.265140,-0.267773,-0.224765,-0.178320,-0.310361,-0.522899,-0.472318,-0.576367
2,59064,16.666667,0.222222,-2.212789,-1.571881,-1.011672,-2.543643,-1.322408,-0.548755,-0.329249,...,-0.570114,-0.613106,-0.272273,-0.274099,-0.224765,-0.178320,-0.353106,-0.601031,-0.489654,-0.602742
3,29202,110.000000,0.498307,-1.199646,0.919876,0.698718,-0.281321,-0.786844,0.884927,1.402131,...,-0.318355,-0.613106,-0.214799,-0.219491,-0.159289,-0.065244,-0.178750,-0.548091,-0.101452,-0.466073
4,57014,113.333333,0.280748,0.221575,-0.524817,0.003025,0.009300,0.302447,-0.548755,-0.329249,...,-0.280209,-0.613106,-0.259735,-0.259785,-0.224765,-0.178320,-0.273428,-0.555746,-0.464393,-0.523618


In [204]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [205]:
df_train = df_train.drop(['zipcode'],axis=1)
df_test = df_test.drop(['zipcode'],axis=1)
x,y = to_xy(df_train,'score')

# Feature Engineering & Prediction

In [206]:
#features = list(cols)[:10]
#features_train = features + ['score']
#df_train = df_train.loc[:,features_train]
#df_test = df_test.loc[:,features]
#x,y = to_xy(df_train,'score')

kf = KFold(5)
oos_y = []
oos_pred = []
fold = 0
for train,test in kf.split(x):
    fold += 1
    print('Fold #{}'.format(fold))
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
    model.add(Dropout(0.01))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.01))
    model.add(Dense(25, 
                kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01),activation='relu'))
    model.add(Dropout(0.01))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=10, verbose=1, mode='auto')
    checkpointer = ModelCheckpoint(filepath="midterm_model1_best", verbose=0, save_best_only=True)
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=0,epochs=600)
    
    pred = model.predict(x_test)
    oos_y.append(y_test)
    oos_pred.append(pred)
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))

oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = metrics.mean_squared_error(oos_pred,oos_y)
print("Final score (RMSE): {}".format(np.sqrt(score)))

Fold #1
Epoch 00128: early stopping
Fold score (RMSE): 0.6189608573913574
Fold #2
Epoch 00087: early stopping
Fold score (RMSE): 0.8195303082466125
Fold #3
Epoch 00056: early stopping
Fold score (RMSE): 0.9781675338745117
Fold #4
Epoch 00110: early stopping
Fold score (RMSE): 0.5732797980308533
Fold #5
Epoch 00110: early stopping
Fold score (RMSE): 1.3417564630508423
Final score (RMSE): 0.909990668296814


In [207]:
df_test = pd.read_csv('test.csv')
idcheck = df_test["id"]
df_test = multi_merge('zipcode','zipcode',df_test,df_full_detail).drop(['id','zipcode'],axis=1)
true_test = df_test.values.astype(np.float32)
pred_test = model.predict(true_test)

final_test_score = np.concatenate(pred_test)

df_test['score'] = final_test_score
df_final = pd.DataFrame( { 'id':idcheck, 'score': df_test['score']})
df_final.to_csv('csv_to_submit_111.csv', index = False)